# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [16]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
from sklearn.metrics import accuracy_score

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [5]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Using downloaded and verified file: ./mnist/MNIST\raw\train-images-idx3-ubyte.gz
Extracting ./mnist/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist/MNIST\raw
0it [00:00, ?it/s]Using downloaded and verified file: ./mnist/MNIST\raw\train-labels-idx1-ubyte.gz
Extracting ./mnist/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist/MNIST\raw
Using downloaded and verified file: ./mnist/MNIST\raw\t10k-images-idx3-ubyte.gz
Extracting ./mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist/MNIST\raw
  0%|          | 0/4542 [00:00<?, ?it/s]C:\Users\12495\AppData\Local\Programs\Python\Python37\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest

Then, we define the model, object function and optimizer that we use to classify.

In [42]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,6,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),  
            nn.MaxPool2d(2, 2) )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU())
        self.fc3 = nn.Linear(84, 10)
 
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x 
    
model = SimpleNet()
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
lr = 0.001
momentum = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

Next, we can start to train and evaluate!

In [41]:
# train and evaluate
train_acc = []
for epoch in range(NUM_EPOCHS):
    model.train()
    acc = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        output = nn.functional.softmax(output, dim=1)
        acc += accuracy_score(labels, output.argmax(axis=1))
    train_acc.append(acc/len(train_loader))
 
 # test
model.eval()
acc = 0
for images, labels in tqdm(test_loader):
    output = model(images)
    output = nn.functional.softmax(output, dim=1)
    
    acc += accuracy_score(output.argmax(axis=1), labels)


print("\nTraining accuracy:")
print(train_acc)
print("Testing accuracy: %.2f%%" % (acc/len(test_loader)*100))


# evaluate
# TODO:calculate the accuracy using traning and testing dataset

100%|██████████| 78/78 [00:05<00:00, 15.03it/s]
Training accuracy:
[0.975143563034188, 0.9769965277777778, 0.9775974893162394, 0.9794170673076923, 0.9800681089743589, 0.980735844017094, 0.981620592948718, 0.9824552617521367, 0.9829894497863247, 0.9835904113247863]
Testing accuracy: 98.54%



#### Q5:
Please print the training and testing accuracy.